In [ ]:
import pandas as pd
import numpy as np
from datetime import date

In [ ]:
import Graph
import Path
import Evolutionary

Testing graph implementation

In [ ]:
graph = Graph.get_graph()

In [ ]:
graph.edges

In [ ]:
graph.vertices

In [ ]:
assert len(graph.edges) == len(graph.vertices)

In [ ]:
graph.get_neighbours('POLAND')

In [ ]:
assert graph.get_neighbours('Czechia') == graph.get_neighbours('Czech_Republic')

In [ ]:
graph.get_neighbours('Czech_Republic')

In [ ]:
graph.get_stats('POLAND')

In [ ]:
assert np.array_equal(graph.get_stats('Czechia'), graph.get_stats('Czech_Republic'))

In [ ]:
graph.get_stats('Poland')

In [ ]:
assert graph.get_active_cases("Poland", date(2020, 3, 10)) == 17

In [ ]:
assert graph.get_active_cases("Poland", date(2020, 3, 16)) == 122

In [ ]:
assert graph.get_active_cases("Poland", date(2020, 3, 15)) == 102